# December 23, 2022
https://adventofcode.com/2022/day/23

In [ ]:
# too low: 2815

In [158]:
fn = "data/23.txt"
with open(fn, "r") as file:
    puz_text = "".join(file.readlines())

In [187]:
class Map:
    prio = ["N", "S", "W", "E"]
    
    def __init__(self, text=None, verbose=0):
        if text is None:
            self.map = None
        else:
            self.map = [ [c for c in line] for line in text.split("\n") ]

        self.blank = "."
        self.elf   = "#"

        # map will have one extra row/col of blanks on each end
        self.map = [ [self.blank] + line + [self.blank] for line in self.map ]
        self.map = [[self.blank] * len(self.map[0])] + self.map + [[self.blank] * len(self.map[0])]
        self.rounds = 0

        self.verbose = verbose

    def copy(self):
        newmap = Map()
        
        return newmap

    # check functions return True if the specific area is clear
    def check_loc(self, x, y):
        if ( x < 0 or
             x >= len(self.map[0]) or
             y < 0 or
             y >= len(self.map) or
             self.map[y][x] == self.blank ):
            return True

    def check_around(self, x, y):
        return ( self.check_loc(x-1, y-1) and
                 self.check_loc(x-1, y  ) and
                 self.check_loc(x-1, y+1) and
                 self.check_loc(x  , y-1) and
                 self.check_loc(x  , y+1) and
                 self.check_loc(x+1, y-1) and
                 self.check_loc(x+1, y  ) and
                 self.check_loc(x+1, y+1) )
    def check_north(self, x, y):
        return ( self.check_loc(x-1, y-1) and
                 self.check_loc(x  , y-1) and
                 self.check_loc(x+1, y-1) )
    def check_south(self, x, y):
        return ( self.check_loc(x-1, y+1) and
                 self.check_loc(x ,  y+1) and
                 self.check_loc(x+1, y+1) )
    def check_west(self, x, y):
        return ( self.check_loc(x-1, y-1) and
                 self.check_loc(x-1, y  ) and
                 self.check_loc(x-1, y+1) )
    def check_east(self, x, y):
        return ( self.check_loc(x+1, y-1) and
                 self.check_loc(x+1, y  ) and
                 self.check_loc(x+1, y+1) )

    def diffuse_once(self):
        W = len(self.map[0])
        H = len(self.map)
        newmap = [line.copy() for line in self.map]

        # DETERMINE ALL ELVES INTENTION
        # represent an elf wanting to move into a spot by recording the direction it moved
        # by construction, no elves on the edge of a map
        stable = True
        for y in range(1,H-1):
            for x in range(1, W-1):
                #print(x,y)
                if self.map[y][x] == self.blank:
                    # nobody here
                    continue
                
                if self.check_around(x,y):
                    if self.verbose > 0:
                        print(f"Elf at {x},{y} wants to stay")
                    continue # stay

                # Found an elf that wants to move. We're not stable
                stable = False

                # otherwise check each cardinal direction
                # exit early if we find a safe spot
                # for safe spots we'll append a movement char so we can check if multiple elves want to move there
                for i in range(4):
                    to_check = Map.prio[ (self.rounds + i) % 4 ]

                    if to_check == "N":
                        safe = self.check_north(x,y)
                        if safe:
                            if self.verbose > 0:
                                print(f"Elf at {x},{y} wants to go ^")

                            newmap[y-1][x] += "^"
                            newmap[y][x] = self.blank
                            break

                    elif to_check == "S":
                        safe = self.check_south(x,y)
                        if safe:
                            if self.verbose > 0:
                                print(f"Elf at {x},{y} wants to go v")

                            newmap[y+1][x] += "v"
                            newmap[y][x] = self.blank
                            break

                    elif to_check == "W":
                        safe = self.check_west(x,y)
                        if safe:
                            if self.verbose > 0:
                                print(f"Elf at {x},{y} wants to go <")

                            newmap[y][x-1] += "<"
                            newmap[y][x] = self.blank
                            break

                    else: # to_check == "E":
                        safe = self.check_east(x,y)
                        if safe:
                            if self.verbose > 0:
                                print(f"Elf at {x},{y} wants to go >")
                            newmap[y][x+1] += ">"
                            newmap[y][x] = self.blank
                            break
                    if i==3 and self.verbose > 0:
                        print(f"Elf at {x},{y} feels crowded")
                # end check all directions
            # end iterate across row
        # end iterate over all rows

        # FINALIZE THE NEW MAP
        expand_north = expand_south = expand_west = expand_east = False
        for y in range(H):
            for x in range(W):
                #print(x,y, newmap[y][x])
                if len(newmap[y][x]) == 1:
                    # nobody tried to move here.
                    # either still a blank spot or a stationary elf
                    continue
                if len(newmap[y][x]) == 2:
                    # one elf tried to move here. It succeeds
                    newmap[y][x] = self.elf
                    # determine if we should expand map
                    if y == 0:
                        expand_north = True
                    elif y == H-1:
                        expand_south = True
                    elif x == 0: # no diagonal moves, so x is not 0 if we're at the top or left edge
                        expand_west = True
                    elif x == W-1:
                        expand_east = True
                    continue

                # multiple elves tried to move here. They fail and go back to original spots
                # Note that original spots must be blank because nobody would try to move where they were
                for c in newmap[y][x][1:]: # note: 0th position is the original blank space
                    if c == "^": # elf moved north, so put it back one space south
                        newmap[y+1][x] = self.elf
                    elif c == "v": # elf moved south, so put it back one space north
                        newmap[y-1][x] = self.elf
                    elif c == ">": # elf moved east, so put it back one space west
                        newmap[y][x-1] = self.elf
                    elif c == "<": # elf moved west, so put it back one space east
                        newmap[y][x+1] = self.elf
                
                # replace with empty tile
                # we know tile is blank because elves don't try to move if there was an elf already there
                newmap[y][x] = self.blank # replace with the empty tile

        # end finalize map

        # POSSIBLY EXPAND MAP
        if expand_east:
            if expand_west:
                newmap = [ [self.blank] + line + [self.blank] for line in newmap]
            else:
                newmap = [line + [self.blank] for line in newmap]
        elif expand_west:
            newmap = [ [self.blank] + line for line in newmap]
        if expand_north:
            newmap = [[self.blank] * len(newmap[0])] + newmap
        if expand_south:
            newmap = newmap + [[self.blank] * len(newmap[0])]

        # AT LAST, UPDATE THE TRUE MAP
        self.map = newmap
        self.rounds += 1
        return stable

    def measure_empty_space(self):
        minx = maxx = miny = maxy = None
        # find boundaries of elf-space
        for y in range(len(self.map)):
            if self.elf in self.map[y]:
                miny = y
                break
        for y in range(len(self.map)-1, -1, -1):
            if self.elf in self.map[y]:
                maxy = y
                break
        for x in range(len(self.map[0])):
            for y in range(len(self.map)):
                if self.map[y][x] == self.elf:
                    minx = x
                    break
            if minx is not None:
                break
        for x in range(len(self.map[0])-1, -1, -1):
            for y in range(len(self.map)):
                if self.map[y][x] == self.elf:
                    maxx = x
                    break
            if maxx is not None:
                break

        count = sum( [sum([c == self.blank for c in line[minx:maxx+1]]) for line in self.map[miny:maxy+1]] )
        return count, [minx, maxx, miny, maxy]

    def diffuse(self):
        while not self.diffuse_once():
            if self.rounds % 100 == 0:
                print(f"Completed {self.rounds} rounds so far.")
        return self.rounds

    def __str__(self):
        return "\n".join( ["".join(line) for line in self.map] )
    def __repr__(self):
        return str(self)


def copy_map(map):
    return [ line.copy() for line in map ]

In [127]:
mini_test = f'''.....
..##.
..#..
.....
..##.
.....'''

mini = Map(mini_test)
print(0)
print(mini, "\n")

for i in range(5):
    print("\n", i+1)
    mini.diffuse_once()
    print(mini, "\n")

0
.......
.......
...##..
...#...
.......
...##..
.......
....... 


 1
.......
...##..
.......
...#...
....#..
...#...
.......
....... 


 2
.......
.......
...##..
..#....
.....#.
.......
...#...
....... 


 3
.......
...#...
.....#.
.#.....
.....#.
.......
...#...
....... 


 4
.......
...#...
.....#.
.#.....
.....#.
.......
...#...
....... 


 5
.......
...#...
.....#.
.#.....
.....#.
.......
...#...
....... 



In [182]:
test_str = f'''..............
..............
.......#......
.....###.#....
...#...#.#....
....#...##....
...#.###......
...##.#.##....
....#..#......
..............
..............
..............'''

test = Map(test_str)
print(0)
print(test, "\n")

for i in range(10):
    print("\n", i+1)
    test.diffuse_once()
    print(test, "\n")

0
................
................
................
........#.......
......###.#.....
....#...#.#.....
.....#...##.....
....#.###.......
....##.#.##.....
.....#..#.......
................
................
................
................ 


 1
................
................
........#.......
......#...#.....
....#..#.#......
........#..#....
.....#.#.##.....
...#..#.#.......
...#.#.#.##.....
................
.....#..#.......
................
................
................ 


 2
................
................
........#.......
.....#.....#....
....#..#.#......
........#...#...
....#..#.#......
..#...#.#.#.....
................
...#.#.#.##.....
.....#..#.......
................
................
................ 


 3
................
................
........#.......
......#....#....
...#..#...#.....
........#...#...
....#..#.#......
..#..#.....#....
........##......
...##.#....#....
....#...........
........#.......
................
................ 


 4
................
.....

In [171]:
test.measure_empty_space()

(110, [2, 13, 1, 11])

In [180]:
puz = Map(puz_text)
#print(0)
#print(puz, "\n")

for i in range(10):
    #print("\n", i+1)
    puz.diffuse_once()
    #print(puz, "\n")
puz.measure_empty_space()

(4162, [1, 84, 2, 84])

### Part 2

In [188]:
mini = Map(mini_test)
mini.diffuse()

4

In [189]:
test = Map(test_str)
test.diffuse()

20

In [190]:
puz = Map(puz_text)
puz.diffuse()

Completed 100 rounds so far.
Completed 200 rounds so far.
Completed 300 rounds so far.
Completed 400 rounds so far.
Completed 500 rounds so far.
Completed 600 rounds so far.
Completed 700 rounds so far.
Completed 800 rounds so far.
Completed 900 rounds so far.


986